<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/images/top.png?raw=true'>

# Configuration (à lancer avant tous les notebooks)

In [1]:
# version de python
import platform
platform.python_version()

'3.8.5'

In [2]:
# la liste des packages installés
!conda list

# packages in environment at /home/lab/anaconda3/envs/azure:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
adal                      1.2.4                    pypi_0    pypi
applicationinsights       0.11.9                   pypi_0    pypi
async_generator           1.10                       py_0    conda-forge
attrs                     20.2.0             pyh9f0ad1d_0    conda-forge
azure-common              1.1.25                   pypi_0    pypi
azure-core                1.8.1                    pypi_0    pypi
azure-graphrbac           0.61.1                   pypi_0    pypi
azure-identity            1.2.0                    pypi_0    pypi
azure-mgmt-authorization  0.61.0                   pypi_0    pypi
azure-mgmt-containerregistry 2.8.0                    pypi_0    pypi
azure-mgmt-keyvault       2.2.0                    pypi_0    pypi
azure-mgmt-resource       10.2.0                   pypi_0    pypi
a

In [3]:
# version de la SDK azureml
import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

Ready to use Azure ML 1.13.0


Si le notebook est executé en dehors d'Azure, il faut télécharger le fichier config.json depuis le portail https://portal.azure.com/, et le mettre dans le workspace qui contient le notebook.

Si le notebook est exécuté directement depuis le workspace Azure, le fichier de config devrait déjà être là.

In [4]:
# connexion au workspace
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, "loaded")

jt-dp100 loaded


# Enregistrement des données

In [5]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'titanic dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/titanic/train.csv'], # Upload the diabetes csv files in /data
                        target_path='titanic-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'titanic-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='titanic dataset',
                                description='titanic data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

Uploading an estimated of 1 files
Uploading ./data/titanic/train.csv
Uploaded ./data/titanic/train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


MemoryError: Engine process terminated. This is most likely due to system running out of memory. Please retry with increased memory. |session_id=06657b47-111c-43e4-b9ec-689d8d88c2d0

# Création d'un environnement (conda)

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create a Python environment for the experiment
titanic_env = Environment("titanic-experiment-env")
titanic_env.python.user_managed_dependencies = False # Let Azure ML manage dependencies
titanic_env.docker.enabled = True # Use a docker container

# Create a set of package dependencies (conda or pip as required)
titanic_packages = CondaDependencies.create(conda_packages=['scikit-learn'],
                                          pip_packages=['azureml-defaults', 'azureml-dataprep[pandas]'])

# Add the dependencies to the environment
titanic_env.python.conda_dependencies = titanic_packages

print(titanic_env.name, 'defined.')

In [ ]:
# Register the environment
titanic_env.register(workspace=ws)

# Création du script

In [ ]:
import os

# Create a folder for the experiment files
experiment_folder = 'titanic_training_logistic'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/titanic_training.py
from azureml.core import Run
import joblib

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

# Get the experiment run context
run = Run.get_context()

# load the diabetes data (passed as an input dataset)
print("Loading Data...")
titanic = run.input_datasets['titanic'].to_pandas_dataframe()


features = ["Age","Pclass","SibSp", "Parch", "Fare","Sex", "Embarked"]

X = pd.get_dummies(train_data[features])
y = train_data["Survived"]

# missing values
imputer = SimpleImputer(strategy='most_frequent')
imputed_X = pd.DataFrame(imputer.fit_transform(X))
imputed_X.columns = X.columns
imputed_X[["Age","Pclass","SibSp", "Parch", "Fare"]] = imputed_X[["Age","Pclass","SibSp", "Parch", "Fare"]].astype('int')


# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(imputed_X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# model
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X_train, y_train)


# score
predictions = model.predict(X_valid)

mae = mean_absolute_error(predictions.astype('int'), y_valid)
acc = accuracy_score(y_valid, predictions.astype('int'))
print("mae : {}, accuracy : {}".format(mae, acc))
run.log('mae', mae)
run.log('acc',acc)

# Save the trained model in the outputs folder
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/titanic_model.pkl')

run.complete()

# Execution du script en local

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment
from azureml.widgets import RunDetails


# Get the training dataset
titanic_ds = ws.datasets.get("titanic dataset")

# Create an estimator
estimator = Estimator(source_directory=experiment_folder,
                      inputs=[titanic_ds.as_named_input('titanic')],
                      compute_target = 'local',
                      environment_definition = titanic_env,
                      entry_script='titanic_training.py')

# Create an experiment
experiment = Experiment(workspace = ws, name = 'titanic-training')

# Run the experiment
run = experiment.submit(config=estimator)
# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

# Execution du script sur un cluster distant

Provisionnement ou récupération d'un cluster existant appelé "aml-cluster". Voir [tutoriel microsoft : partie 1](https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-train-models-with-aml#train-on-a-remote-cluster)

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "aml-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                                min_nodes=compute_min_nodes,
                                                                max_nodes=compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(
        ws, compute_name, provisioning_config)

    # can poll for a minimum number of nodes and for a specific timeout.
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20)

    # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

Même estimator que dans la partie précédente mais avec le cluster dans la compute_target. Il va falloir provisionner le cluster (normalement 0 noeuds actifs initialement), cela peut prendre un peu de temps...

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment
from azureml.widgets import RunDetails


# Get the training dataset
titanic_ds = ws.datasets.get("titanic dataset")

# Create an estimator
estimator = Estimator(source_directory=experiment_folder,
                      inputs=[titanic_ds.as_named_input('titanic')],
                      compute_target = compute_target,
                      environment_definition = titanic_env,
                      entry_script='titanic_training.py')

# Create an experiment
experiment = Experiment(workspace = ws, name = 'titanic-training')

# Run the experiment
run = experiment.submit(config=estimator)
# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

# Ne pas oublier à la fin de l'expérience!!
(si votre travail à utilisé une instance de calcul)

<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/images/down.png?raw=true'>



In [ ]:
# stop toutes les instances de calcul
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance and compute.get_status().state == 'Running':
        print('try to stop compute', compute.name)
        compute.stop(show_output=True)

In [ ]:
# liste tous les compute pour vérifier qu'elles sont éteintes
for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance:
        print(compute.name, compute.get_status())

# Ressources

[api azure](https://docs.microsoft.com/en-us/python/api/azureml-core)

[parcours d'apprentissage microsoft](https://docs.microsoft.com/fr-fr/learn/paths/build-ai-solutions-with-azure-ml-service/)

[le repository microsoft](https://github.com/MicrosoftDocs/mslearn-aml-labs.git)